In [19]:
!pip install proxmin

  Using cached proxmin-0.6.10-py2.py3-none-any.whl (18 kB)


In [20]:
import constant as c
import functions as f
import pandas as pd
import os
from os.path import join
from itertools import combinations
import numpy as np
from scipy.spatial.distance import euclidean, hamming
import collections

from numpy import matlib

import proxmin
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.preprocessing import StandardScaler

from joblib import dump, load



%matplotlib inline


In [23]:
ant_dataframe = []
camel_dataframe = []
ivy_dataframe = []
jedit_dataframe = []
lucene_dataframe = []
poi_dataframe = []
synapse_dataframe = []
velocity_dataframe = []
xalan_dataframe = []
xerces_dataframe = []
log4j_dataframe = []


for filename in os.listdir(c.ant):
    tmp = pd.read_csv(join(c.ant, filename))
    ant_dataframe.append(tmp)

for filename in os.listdir(c.camel):
    tmp = pd.read_csv(join(c.camel, filename))
    camel_dataframe.append(tmp)
    
for filename in os.listdir(c.ivy):
    tmp = pd.read_csv(join(c.ivy, filename))
    ivy_dataframe.append(tmp)   
    
for filename in os.listdir(c.jedit):
    tmp = pd.read_csv(join(c.jedit, filename))
    jedit_dataframe.append(tmp)
    
for filename in os.listdir(c.lucene):
    tmp = pd.read_csv(join(c.lucene, filename))
    lucene_dataframe.append(tmp)
    
for filename in os.listdir(c.poi):
    tmp = pd.read_csv(join(c.poi, filename))
    poi_dataframe.append(tmp)
    
for filename in os.listdir(c.synapse):
    tmp = pd.read_csv(join(c.synapse, filename))
    synapse_dataframe.append(tmp)    
    
for filename in os.listdir(c.velocity):
    tmp = pd.read_csv(join(c.velocity, filename))
    velocity_dataframe.append(tmp)   
    
for filename in os.listdir(c.xalan):
    tmp = pd.read_csv(join(c.xalan, filename))
    xalan_dataframe.append(tmp)
    
for filename in os.listdir(c.xerces):
    tmp = pd.read_csv(join(c.xerces, filename))
    xerces_dataframe.append(tmp)

for filename in os.listdir(c.log4j):
    tmp = pd.read_csv(join(c.log4j, filename))
    log4j_dataframe.append(tmp)

    

ant_dataframe = pd.concat(ant_dataframe)
camel_dataframe = pd.concat(camel_dataframe)
ivy_dataframe = pd.concat(ivy_dataframe)
jedit_dataframe = pd.concat(jedit_dataframe)
lucene_dataframe = pd.concat(lucene_dataframe)
poi_dataframe = pd.concat(poi_dataframe)
synapse_dataframe = pd.concat(synapse_dataframe)
velocity_dataframe = pd.concat(velocity_dataframe)
xalan_dataframe = pd.concat(xalan_dataframe)
xerces_dataframe = pd.concat(xerces_dataframe)
log4j_dataframe = pd.concat(log4j_dataframe)

In [24]:
ant_dataframe[ant_dataframe.version == 1.4]

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.4,org.apache.tools.ant.taskdefs.PathConvert,14,3,0,8,41,33,1,...,1.000000,2,0.740000,0.357143,1,1,25.857143,4,1.5714,0
1,ant,1.4,org.apache.tools.ant.taskdefs.Untar,5,3,0,9,43,0,0,...,1.000000,0,0.902439,0.533333,2,3,53.600000,1,0.8000,0
2,ant,1.4,org.apache.tools.ant.taskdefs.PumpStreamHandler,13,1,1,8,20,46,6,...,1.000000,0,0.000000,0.615385,0,0,7.230769,1,0.7692,0
3,ant,1.4,org.apache.tools.ant.taskdefs.Copydir,8,4,0,7,35,4,0,...,1.000000,0,0.917647,0.468750,3,3,31.375000,5,1.3750,0
4,ant,1.4,org.apache.tools.tar.TarEntry,31,1,0,6,65,243,4,...,1.000000,0,0.000000,0.182796,1,1,22.612903,4,1.1613,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,5,3,0,7,19,4,4,...,0.000000,0,0.902439,0.400000,1,1,28.800000,1,0.8000,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,17,1,3,15,66,90,4,...,1.000000,2,0.000000,0.197917,0,0,35.882353,14,2.4118,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,23,1,0,31,82,201,22,...,0.444444,0,0.000000,0.318182,0,0,45.739130,24,2.3043,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,11,1,8,22,86,15,8,...,1.000000,7,0.000000,0.266667,0,0,77.727273,22,4.6364,1


In [4]:
needed = ['name', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features_withbug = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc']


distances = ['e', 'm' , 'c']

In [11]:
log4j_dataframe[log4j_dataframe.version == 1.2]

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,log4j,1.2,org.apache.log4j.chainsaw.XMLFileHandler,8,2,0,4,22,4,1,...,1.000000,2,0.708333,0.291667,1,2,31.125000,6,2.3750,3
1,log4j,1.2,org.apache.log4j.spi.Configurator,1,1,0,7,1,0,6,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,2
2,log4j,1.2,org.apache.log4j.nt.test.NTMin,7,1,0,6,30,17,0,...,0.000000,1,0.000000,0.277778,0,0,19.285714,2,0.8571,1
3,log4j,1.2,org.apache.log4j.or.DefaultRenderer,2,1,0,2,4,1,1,...,0.000000,0,0.000000,0.750000,0,0,3.000000,1,0.5000,2
4,log4j,1.2,org.apache.log4j.varia.FallbackErrorHandler,8,1,0,6,22,22,0,...,0.000000,2,0.000000,0.339286,0,0,18.250000,2,1.1250,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,log4j,1.2,org.apache.log4j.test.MDCStress,10,2,0,6,38,9,0,...,0.000000,2,0.884058,0.222222,0,0,30.600000,4,2.0000,1
201,log4j,1.2,org.apache.log4j.helpers.DateLayout,11,2,1,7,27,19,1,...,0.555556,0,0.444444,0.303030,0,0,16.181818,7,2.0000,2
202,log4j,1.2,org.apache.log4j.net.test.SocketMin,8,1,0,7,42,14,0,...,0.000000,2,0.000000,0.238095,0,0,37.000000,4,1.5000,2
203,log4j,1.2,org.apache.log4j.test.X1Logger,7,3,0,5,19,7,1,...,0.333333,1,0.912281,0.500000,2,2,9.142857,1,0.7143,0


In [37]:
all_dataframe = [
                ant_dataframe,
                camel_dataframe,
                ivy_dataframe,
                jedit_dataframe, 
                lucene_dataframe,
                poi_dataframe, 
                synapse_dataframe, 
                velocity_dataframe, 
                xalan_dataframe, 
                #xerces_dataframe,  #VA PRIMA ORDINATO (DA RICORDARE)
                log4j_dataframe
              ]

In [34]:
ant_dataframe
versions = ant_dataframe.version.unique()
names = ant_dataframe.name.unique()
version_tuple = [(x, y) for x, y in zip(versions[0::1],versions[1::1])]

In [42]:
version_tuple
for v in version_tuple:
    print("__________________________")
    previous = ant_dataframe.loc[ant_dataframe.version == v[0]]
    current = ant_dataframe.loc[ant_dataframe.version == v[1]]
    previous = previous.sort_values('name').reset_index()
    current = current.sort_values('name').reset_index()
    previous = previous.drop(['index'], axis = 1)
    current = current.drop(['index'], axis = 1)
    modules_previous = previous.name.unique()
    modules_current = current.name.unique()
    for k in distances[]:
        print("PREVIOUS:", previous.version[0])
        print("CURRENT:", current.version[0])
        print("DISTANCE:", k)
        D = f.create_D(current, previous, features, k)
        idx = f.runDS3(D, reg = .5, verbose = False)

__________________________
__________________________
__________________________
__________________________


In [13]:
for d in all_dataframe:
    
    print("__________________________")
    print(d['project'].unique())
    versions = d.version.unique()
    names = d.name.unique()
    version_tuple = [(x, y) for x, y in zip(versions[0::1],versions[1::1])]
    for v in version_tuple:
        print("__________________________")
        previous = d.loc[d.version == v[0]]
        current = d.loc[d.version == v[1]]
        previous = previous.sort_values('name').reset_index()
        current = current.sort_values('name').reset_index()
        previous = previous.drop(['index'], axis = 1)
        current = current.drop(['index'], axis = 1)
        modules_previous = previous.name.unique()
        modules_current = current.name.unique()
        for k in distances[0]:
            print("PREVIOUS:", previous.version[0])
            print("CURRENT:", current.version[0])
            print("DISTANCE:", k)
            D = f.create_D(current, previous, features, k)
            idx = f.runDS3(D, reg = .5, verbose = False)
            print('______WITH DS3_____')
            f.run_logisticRegression(previous, current, idx, ds3 = True, verbose = False, plot = False )
            print('______WITHOUT DS3______')
            f.run_logisticRegression(previous, current, None, ds3 = False, verbose = False, plot = False )
            #break

        print("__________________________")
    #break




__________________________
['ant']
__________________________
PREVIOUS: 1.3
CURRENT: 1.4
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.7415730337078652
F-Measure :  0.7415730337078652
G-Mean : 0.8232206329894024
Balance : 0.80737991205058
______WITHOUT DS3______
- LogisticRegression score: 0.7359550561797753
F-Measure :  0.7359550561797754
G-Mean : 0.8192557510887808
Balance : 0.8031925188342882
__________________________
__________________________
PREVIOUS: 1.4
CURRENT: 1.5
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.8668941979522184
F-Measure :  0.8668941979522184
G-Mean : 0.8995553758372629
Balance : 0.8947706239363764
______WITHOUT DS3______
- LogisticRegression score: 0.8464163822525598
F-Measure :  0.8464163822525598
G-Mean : 0.8839788103789794
Balance : 0.8785814891573575
__________________________
__________________________
PREVIOUS: 1.5
CURRENT: 1.6
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.7464387464387464
F-Measure :  0.7464387464387463
G-Mean : 0.8501648389618534
Balance : 0.8193098062075796
______WITHOUT DS3______
- LogisticRegression score: 0.7407407407407407
F-Measure :  0.7407407407407407
G-Mean : 0.8466021815305526
Balance : 0.8152493524144915
__________________________
__________________________
PREVIOUS: 1.6
CURRENT: 1.7
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:31: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


KeyboardInterrupt: 

In [14]:
xerces_dataframe

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,1.2.0,org.xml.sax.SAXException,6,3,3,0,10,0,0,...,1.000000,0,0.850000,0.555556,1,2,7.833333,3,1.0000,0
1,xerces,1.2.0,org.apache.xerces.parsers.DOMParser,63,2,3,27,225,793,7,...,0.896552,6,0.678947,0.152074,2,11,88.698413,6,1.6984,0
2,xerces,1.2.0,org.apache.xerces.dom.RangeExceptionImpl,1,5,0,1,2,0,1,...,0.000000,0,1.000000,1.000000,0,0,5.000000,0,0.0000,1
3,xerces,1.2.0,org.apache.html.dom.HTMLIFrameElementImpl,21,4,0,2,25,210,0,...,0.000000,0,0.824561,0.523810,0,0,4.714286,1,0.9524,0
4,xerces,1.2.0,org.apache.xerces.dom.ChildAndParentNode,25,3,3,13,67,34,3,...,0.833333,3,0.684932,0.205000,2,22,43.560000,11,2.2800,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,xerces,init,org.apache.xerces.dom.DeferredElementDefinitio...,4,3,0,5,11,0,1,...,0.500000,0,0.940000,0.500000,2,12,17.000000,2,1.0000,0
158,xerces,init,org.apache.xerces.utils.XMLMessageProvider,3,1,0,5,3,3,5,...,0.000000,0,0.000000,0.583333,0,0,0.000000,1,1.0000,0
159,xerces,init,org.xml.sax.ErrorHandler,3,1,0,0,3,3,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,2
160,xerces,init,org.apache.xerces.utils.XMLMessages,5,1,0,2,14,0,1,...,0.009404,0,0.000000,0.500000,0,0,155.600000,9,2.2000,0
